In [2]:
from os import listdir, path, mkdir
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import datetime as dt
import requests
from tqdm import tqdm
import os
import glob
import pandas as pd
from wps_formatter import WRFInputFormatter

In [3]:
def get_file_by_url(url, file_name, overwrite=False):
    if path.exists(file_name) and not overwrite:
        return
    response = requests.get(url)
    with open(file_name, 'wb') as f:
        f.write(response.content)

In [4]:
def create_dir(path_to_dir):
    if path_to_dir == '':
        return
    if path.exists(path_to_dir):
        return
    else:
        mkdir(path_to_dir)

In [5]:
def clear_dir(path):
    files = glob.glob(f'{path}/*')
    for f in files:
        os.remove(f)

In [10]:
def download_reanalysis_year_spreads(year, local_path='data', verbose=False, overwrite=False):
    #define a bunch of constants refering to where these datasets are stored on the NOAA servers
    year_id = 'SI' if year < 1981 else 'MO'
    BASE_URL = 'https://downloads.psl.noaa.gov/Datasets/20thC_ReanV3'
    SPREADS_URL = 'https://downloads.psl.noaa.gov/Datasets/20thC_ReanV3/spreads'
    
    DATA_PATHS = [f'accums{year_id}/apcp.{year}.nc', f'sfc{year_id}/prate.{year}.nc', f'accums{year_id}/snod.{year}.nc',
                  f'subsfc{year_id}/soilw.{year}.nc', f'accums{year_id}/runoff.{year}.nc']
                  #f'prs{year_id}/hgt.{year}.nc', f'prs{year_id}/uwnd.{year}.nc', f'prs{year_id}/vwnd.{year}.nc']
    
    #download everything save the surface variables
    create_dir(f'{local_path}/{year}')
    create_dir(f'{local_path}/{year}/means')
    create_dir(f'{local_path}/{year}/spreads')
    for path_file in DATA_PATHS:
        if verbose:
                print(f'Downloading {path_file}')
        get_file_by_url(f'{BASE_URL}/{path_file}', f'{local_path}/{year}/means/{path_file.split("/")[1]}', overwrite=overwrite)
        get_file_by_url(f'{SPREADS_URL}/{path_file}', f'{local_path}/{year}/spreads/{path_file.split("/")[1]}', overwrite=overwrite)

In [7]:
def download_reanalysis_year(year, local_path='data', verbose=False, overwrite=False):
    #define a bunch of constants refering to where these datasets are stored on the NOAA servers
    year_id = 'SI' if year < 1981 else 'MO'
    DATA_SUBSETS = ['mean_sea_level', 'pressure_levels', 'subsurface', 'tropopause', 'two_meters', 'ten_meters']
    DATA_PATHS   = [f'misc{year_id}', f'prs{year_id}', f'subsfc{year_id}', f'tropo{year_id}', f'2m{year_id}', f'10m{year_id}']
    BASE_URL = 'https://downloads.psl.noaa.gov/Datasets/20thC_ReanV3'
    #SPREADS_URL = 'https://downloads.psl.noaa.gov/Datasets/20thC_ReanV3/spreads'

    MSL_FILES = [f'prmsl.{year}.nc']
    PL_FILES = [f'air.{year}.nc', f'hgt.{year}.nc', f'rhum.{year}.nc', f'uwnd.{year}.nc', f'vwnd.{year}.nc']
    SS_FILES = [f'soilw.{year}.nc', f'tsoil.{year}.nc']
    TP_FILES = [f'air.tropo.{year}.nc', f'hgt.tropo.{year}.nc', f'pres.tropo.{year}.nc', 
                f'uwnd.tropo.{year}.nc', f'vwnd.tropo.{year}.nc']
    TM_FILES = [f'air.2m.{year}.nc', f'rhum.2m.{year}.nc']
    TEN_FILES = [f'uwnd.10m.{year}.nc', f'vwnd.10m.{year}.nc']
    
    MASTER_FILE_LIST = [MSL_FILES, PL_FILES, SS_FILES, TP_FILES, TM_FILES, TEN_FILES]
    
    #unlike the others, the surface variables are in a couple of different repositories, so they are accessed differently
    SURFACE_PATHS = [f'timeInvariant{year_id}/land.nc', f'sfc{year_id}/pres.sfc.{year}.nc', 
                     f'sfc{year_id}/skt.{year}.nc', f'accums{year_id}/snod.{year}.nc', f'timeInvariant{year_id}/hgt.sfc.nc']
    
    #download everything save the surface variables
    create_dir(f'{local_path}/{year}')
    create_dir(f'{local_path}/{year}/means')
    create_dir(f'{local_path}/{year}/spreads')
    for directory, noaa_path, file_list in zip(DATA_SUBSETS, DATA_PATHS, MASTER_FILE_LIST):
        create_dir(f'{local_path}/{year}/means/{directory}')
        for file in file_list:
            if verbose:
                print(file)
            get_file_by_url(f'{BASE_URL}/{noaa_path}/{file}', f'{local_path}/{year}/means/{directory}/{file}', overwrite=overwrite)
                   
    
    #and now for the surface variables
    create_dir(f'{local_path}/{year}/means/surface')
    create_dir(f'{local_path}/{year}/spreads/surface')
    for path_file in SURFACE_PATHS:
        get_file_by_url(f'{BASE_URL}/{path_file}', f'{local_path}/{year}/means/surface/{path_file.split("/")[1]}', overwrite=overwrite)
                   
        
    #finally, we've got change the names/variables names of some files so the import program
    #can work with it
    update_list = {f'{local_path}/{year}/means/surface/hgt.sfc.nc': 'hgtsfc', 
                   f'{local_path}/{year}/means/surface/pres.sfc.{year}.nc': 'psfc',
                   f'{local_path}/{year}/means/ten_meters/uwnd.10m.{year}.nc': 'uas',
                   f'{local_path}/{year}/means/ten_meters/vwnd.10m.{year}.nc': 'vas',
                   f'{local_path}/{year}/means/tropopause/air.tropo.{year}.nc': 'airtrp',
                   f'{local_path}/{year}/means/tropopause/hgt.tropo.{year}.nc': 'hgttrp',
                   f'{local_path}/{year}/means/tropopause/pres.tropo.{year}.nc': 'prestrp',
                   f'{local_path}/{year}/means/tropopause/uwnd.tropo.{year}.nc': 'uwndtrp',
                   f'{local_path}/{year}/means/tropopause/vwnd.tropo.{year}.nc': 'vwndtrp',
                   f'{local_path}/{year}/means/two_meters/air.2m.{year}.nc': 'tas',
                   f'{local_path}/{year}/means/two_meters/rhum.2m.{year}.nc': 'rhums'
                  }
    for file in update_list:
        new_name = f'{file[:file.rfind("/")]}/{update_list[file]}.nc'
        old_var = file[file.rfind('/')+1:].split('.')[0]
        
        if path.exists(file):
            old_dataset = xr.open_dataset(file)
            new_dataset = old_dataset.rename({old_var: update_list[file]})
            new_dataset.to_netcdf(new_name)

            old_dataset.close()
            os.remove(file)

In [13]:
years = [1840, 1841, 1842, 1843, 1845, 1846, 1847, 1848]
for year in years:
    print(year)
    download_reanalysis_year_spreads(year, local_path = 'D:/data', verbose=True)

1840
1841
1842
1843
1845
1846
1847
1848


In [7]:
#right now it can't handle the tropopause and 2 meter datasets with the same variable name easily, need to fix
def generate_variable_list(data_path, year, grib_codes):
    variable_paths = {}
    for level_type in os.listdir(f'{data_path}/{year}/means'):
        if level_type == '.ipynb_checkpoints' or '.nc' in level_type:
            continue
        for variable_file in os.listdir(f'{data_path}/{year}/means/{level_type}'):
            if variable_file == '.ipynb_checkpoints':
                continue
            var_name = variable_file.split('.')[0]
            if var_name in variable_paths:
                continue
            variable_paths[var_name] = f'{data_path}/{year}/means/{level_type}/{variable_file}'
            if var_name not in grib_codes.index:
                print(f'{var_name} must be added to grib codes table')
    return variable_paths

In [8]:
def add_row_to_grib_codes(grib_codes_list, var_name, description, grib_code, 
                          level_code, metgrid_name, metgrid_units, 
                          save_path=None, overwrite=False):
    grib_codes_list.loc[var_name] = [description, grib_code, level_code, metgrid_name, metgrid_units]

In [70]:
grib_codes = pd.read_pickle('grib_codes.df.pkl')
add_row_to_grib_codes(grib_codes, 'prmsl', 'Pressure reduced to mean sea level', '2', '102', 'PMSL', 'Pa')
add_row_to_grib_codes(grib_codes, 'air', 'Air Temperature', '11', '', 'TT', 'K')
add_row_to_grib_codes(grib_codes, 'hgt', 'Geopotential Height', '7', '', 'HGT', 'm')
add_row_to_grib_codes(grib_codes, 'rhum', 'Relative Humidity', '52', '', 'RH', '%')
add_row_to_grib_codes(grib_codes, 'uwnd', 'U Wind', '33', '', 'UU', 'm s-1')
add_row_to_grib_codes(grib_codes, 'vwnd', 'V Wind', '34', '', 'VV', 'm s-1')
add_row_to_grib_codes(grib_codes, 'land', 'Land/Sea Flag', '81', '1', 'LANDSEA', 'proprtn')
add_row_to_grib_codes(grib_codes, 'pres', 'Surface Pressure', '1', '1', 'PSFC', 'Pa')
add_row_to_grib_codes(grib_codes, 'skt', 'Skin Temperature', '11', '1', 'SKINTEMP', 'K')
add_row_to_grib_codes(grib_codes, 'snod', 'Snow Depth', '66', '1', 'SNOWH', 'm')
add_row_to_grib_codes(grib_codes, 'hgtsfc', 'Terrain Height', '7', '1', 'SOILHGT', 'm')
add_row_to_grib_codes(grib_codes, 'airtrp', 'Air Temperature at Tropopause', '2', '7', 'TTROP', 'K')
add_row_to_grib_codes(grib_codes, 'hgttrp', 'Geopotential Height at Tropopause', '7', '7', 'HGTTROP', 'm')
add_row_to_grib_codes(grib_codes, 'prestrp', 'Pressure at Tropopause', '2', '7', 'PTROP', 'Pa')
add_row_to_grib_codes(grib_codes, 'uwndtrp', 'U Wind at Tropopause', '2' , '7', 'UTROP', 'm s-1')
add_row_to_grib_codes(grib_codes, 'vwndtrp', 'V Wind at Tropopause', '2' , '7', 'VTROP', 'm s-1')
add_row_to_grib_codes(grib_codes, 'rhums', 'Relative Humidity at 2 Meters', '52' , '105', 'RH', '%')

In [10]:
variable_paths = generate_variable_list('data', 1993, grib_codes)

params = {
    'variable_paths' : variable_paths,
    'variable_table' : grib_codes,
    
    # Subset to this time range (start, end)
    'time_range' : (dt.datetime(1993,3,21,0), dt.datetime(1993,3,22,12)),
    
    # We want output every this many hours.  This particular
    # string format is what xarray looks for
    'time_freq' : '3H',
    
    # Set the vertical coordinate type of the raw dataset.
    # Options are 'pressure', with limited support for 'hybrid-p' and 'hybrid-z'
    'vcoord_type' : 'pressure',
    
    # These are the names of the vertical coordinates in the datasets
    # For upper air pressure levels:
    'vcoord' : 'level',
    
    # And for subsurface soil levels
    'soilcoord' : 'level'
}

FileNotFoundError: [Errno 2] No such file or directory: 'data/1993/means'

In [13]:
wrfgen = WRFInputFormatter(**params)

2022-11-21 09:53:36,893 - WRFInputFormatter - INFO - THESE ARE GEOG BOUNDS: None
2022-11-21 09:53:36,894 - WRFInputFormatter - INFO - Building domain structure from pressure field
2022-11-21 09:53:36,895 - WRFInputFormatter - INFO - Loading and subsetting air
2022-11-21 09:53:36,896 - WRFInputFormatter - INFO - Loading variable air from: data/1993/means/pressure_levels/air.1993.nc
2022-11-21 09:53:37,662 - WRFInputFormatter - INFO - Data already at requested frequency of 3H
2022-11-21 09:53:38,065 - WRFInputFormatter - WARNING - Suspect we have hPa...converting to Pa
2022-11-21 09:53:38,733 - WRFInputFormatter - INFO - Done with pressure
2022-11-21 09:53:38,734 - WRFInputFormatter - INFO - GEOG BOUNDS REQD: None
2022-11-21 09:53:38,734 - WRFInputFormatter - INFO - ACTUAL LATS: -90.0 -- 90.0
2022-11-21 09:53:38,735 - WRFInputFormatter - INFO - ACTUAL LONS: 0.0 -- 359.0
2022-11-21 09:53:38,738 - WRFInputFormatter - INFO - Done with initialization


In [14]:
wrfgen.open_WPS_files(outpath = '/home/bparazin/WRF/WPS-4.4')

In [15]:
for var in list(variable_paths.keys()):
    # Load and subset this variable based on 
    # our spatial and temporal dimensions
    # Returns an xarray.DataArray, so this can
    # be looked at and plotted
    varray = wrfgen.load_and_subset(var)
    
    if var in ['air', 'hgt', 'uwnd', 'vwnd']:
        varray = varray[:,:21]
    
    # Soil variables need
    # additional processing
    if var in ['tsoil','soilw']:
        varray = wrfgen.process_soil_levels(varray) 
        #print(varray)
    wrfgen.add_to_WPS(varray)

2022-11-21 09:53:40,750 - WRFInputFormatter - INFO - Loading and subsetting prmsl
2022-11-21 09:53:40,752 - WRFInputFormatter - INFO - Loading variable prmsl from: data/1993/means/mean_sea_level/prmsl.1993.nc
2022-11-21 09:53:41,449 - WRFInputFormatter - INFO - Data already at requested frequency of 3H
2022-11-21 09:53:41,449 - WRFInputFormatter - INFO - Writing prmsl to WPS files
2022-11-21 09:53:41,455 - WRFInputFormatter - INFO -    1993-03-21 00:00:00
2022-11-21 09:53:41,474 - WRFInputFormatter - INFO -    1993-03-21 03:00:00
2022-11-21 09:53:41,490 - WRFInputFormatter - INFO -    1993-03-21 06:00:00
2022-11-21 09:53:41,508 - WRFInputFormatter - INFO -    1993-03-21 09:00:00
2022-11-21 09:53:41,528 - WRFInputFormatter - INFO -    1993-03-21 12:00:00
2022-11-21 09:53:41,546 - WRFInputFormatter - INFO -    1993-03-21 15:00:00
2022-11-21 09:53:41,563 - WRFInputFormatter - INFO -    1993-03-21 18:00:00
2022-11-21 09:53:41,581 - WRFInputFormatter - INFO -    1993-03-21 21:00:00
2022-11-

In [16]:
wrfgen.close_WPS_files()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
ax.add_feature(cartopy.feature.LAKES, alpha=0.5)
ax.add_feature(cartopy.feature.RIVERS)
ax.set_extent([-65, -90, 20, 55], crs=ccrs.PlateCarree())
#im = ax.pcolor(lon, lat, pert_geo)
#plt.colorbar(im)

In [ ]:
listdir('/home/bparazin/WRF/WPS-4.4')

In [ ]:
uwnd = xr.open_dataarray('data/1993/pressure_levels/uwnd.1993.nc')

In [ ]:
uwnd[:,:21].to_netcdf('data/1993/pressure_levels/uwnd.trim.1993.nc')

In [8]:
check = xr.open_dataset('data/1993/means/subsurface/soilw.1993.nc')


In [ ]:
check.close()

In [32]:
os.path.realpath('example_usage.ipynb')

'/mnt/c/Users/bpara/Documents/Git/1844redux/example_usage.ipynb'

In [33]:
os.listdir('/mnt/')

['wsl', 'wslg', 'c']

In [10]:
check['level']

<xarray.DataArray 'level' (level: 4)>
array([  0.,  10.,  40., 100.], dtype=float32)
Coordinates:
  * level    (level) float32 0.0 10.0 40.0 100.0
Attributes:
    units:               cm
    long_name:           Level
    actual_range:        [  0. 100.]
    standard_name:       level
    axis:                Z
    coordinate_defines:  top of layer

In [45]:
nc = xr.open_dataset('/home/bparazin/WRF/WRFV4.4.1/test/em_real/met_em.d01.1993-03-22_12:00:00.nc')

True

In [61]:
for var in nc:
    if var != 'Times' and np.isnan(nc[var].data).any():
        print(var)

SM
ST
SNOWH
CANWAT
ST100200
ST040100
ST010040
ST000010
SM100200
SM040100
SM010040
SM000010


In [69]:
nc['CANWAT'].data

array([[[      nan, 0.       , 0.       , ...,       nan,       nan,
               nan],
        [      nan, 0.       , 0.       , ...,       nan,       nan,
               nan],
        [      nan,       nan, 0.       , ...,       nan,       nan,
               nan],
        ...,
        [0.4849991, 0.5      , 0.5      , ..., 0.5      , 0.5      ,
         0.5      ],
        [0.5      , 0.5      , 0.5      , ..., 0.5      , 0.5      ,
         0.5      ],
        [0.5      , 0.5      , 0.5      , ..., 0.5      , 0.5      ,
         0.5      ]]], dtype=float32)